### Load tensorflow

In [1]:
import tensorflow as tf

### Enable Eager Execution if you are using tensorflow 1.x

In [2]:
#Enable Eager Execution
tf.enable_eager_execution()

In [3]:
import os
os.chdir("E:\PGP AIML\Labs\Residency 6") 
os.getcwd()

'E:\\PGP AIML\\Labs\\Residency 6'

### Collect Data

In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [6]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [7]:
data = data.drop(['date'], axis=1)
data = data.drop(['symbol'], axis=1)

In [8]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [9]:
data.shape

(851264, 5)

### Consider only first 1000 rows in the dataset for building feature set and target set

In [10]:
df = data.head(1000)

In [11]:
df.shape

(1000, 5)

### Convert Float64 to Float32

In [12]:
df = df.astype('float32')

In [13]:
df.dtypes

open      float32
close     float32
low       float32
high      float32
volume    float32
dtype: object

In [14]:
# Assigning the label Column to targetVariable 

targetVariable = 'close'

In [15]:
# Create Input Feature DataFrame X and Output Label Y for the Model Training

X=df.iloc[:,df.columns!=targetVariable]
Y=df.iloc[:,df.columns==targetVariable]

### Divide the data into train and test sets

In [16]:
# Train Test Split

from sklearn.model_selection import train_test_split

np.random.seed(123)
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=123)

In [17]:
y_train.shape

(700, 1)

### Normalize Train and Test Data 

In [18]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
x_train3 = transformer.fit_transform(x_train)

In [19]:
transformer = Normalizer()
x_test3 = transformer.fit_transform(x_test)

In [20]:
y_train3 = y_train.to_numpy()

In [21]:
y_test3 = y_test.to_numpy()

In [22]:
x_train3

array([[1.6141461e-06, 1.6070699e-06, 1.6235808e-06, 1.0000000e+00],
       [1.9083651e-04, 1.8943973e-04, 1.9108210e-04, 9.9999988e-01],
       [4.2019121e-05, 4.1986710e-05, 4.2578187e-05, 1.0000000e+00],
       ...,
       [2.3780945e-05, 2.3505878e-05, 2.4289406e-05, 1.0000000e+00],
       [1.7972880e-05, 1.7853366e-05, 1.8119972e-05, 1.0000000e+00],
       [3.1503550e-05, 3.1358217e-05, 3.1951997e-05, 1.0000000e+00]],
      dtype=float32)

In [23]:
x_test3

array([[2.0255553e-04, 2.0023808e-04, 2.0276188e-04, 9.9999988e-01],
       [4.0810890e-04, 4.0743581e-04, 4.1102557e-04, 9.9999982e-01],
       [1.4952428e-04, 1.4759667e-04, 1.5372543e-04, 9.9999994e-01],
       ...,
       [8.7775479e-06, 8.7052340e-06, 8.8498628e-06, 1.0000000e+00],
       [3.8944449e-06, 3.8684680e-06, 3.9160932e-06, 1.0000000e+00],
       [4.0102986e-04, 3.9948497e-04, 4.0231727e-04, 9.9999982e-01]],
      dtype=float32)

## Building the graph in tensorflow

2.Define Weights and Bias

In [24]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [25]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [26]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [27]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [28]:
#Train for 100 Steps
for i in range(100):
    
    w, b = train(x_train3, y_train3, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, 
          loss(y_train3, prediction(x_train3, w, b)).numpy())

Current Loss on iteration 0 8343.274
Current Loss on iteration 1 8001.9536
Current Loss on iteration 2 7687.392
Current Loss on iteration 3 7397.4927
Current Loss on iteration 4 7130.3223
Current Loss on iteration 5 6884.0957
Current Loss on iteration 6 6657.1733
Current Loss on iteration 7 6448.0435
Current Loss on iteration 8 6255.31
Current Loss on iteration 9 6077.6855
Current Loss on iteration 10 5913.9873
Current Loss on iteration 11 5763.1216
Current Loss on iteration 12 5624.086
Current Loss on iteration 13 5495.9487
Current Loss on iteration 14 5377.8584
Current Loss on iteration 15 5269.028
Current Loss on iteration 16 5168.7275
Current Loss on iteration 17 5076.2915
Current Loss on iteration 18 4991.1016
Current Loss on iteration 19 4912.5923
Current Loss on iteration 20 4840.236
Current Loss on iteration 21 4773.554
Current Loss on iteration 22 4712.099
Current Loss on iteration 23 4655.462
Current Loss on iteration 24 4603.2666
Current Loss on iteration 25 4555.162
Current

### Get the shapes and values of W and b

In [29]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[1.0463421e-02]
 [1.0367184e-02]
 [1.0537945e-02]
 [3.3785690e+01]]
Bias:
 [33.785698]


In [30]:
w.shape

TensorShape([Dimension(4), Dimension(1)])

In [31]:
b.shape

TensorShape([Dimension(1)])

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [32]:
#Initialize Sequential Graph (model)
model = tf.keras.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [33]:
model.fit(x_train3, y_train3, epochs=100)

Epoch 1/100
700/700 [==============================] - 0s 156us/sample - loss: 6259.7098
Epoch 2/100
700/700 [==============================] - 0s 29us/sample - loss: 4353.1224
Epoch 3/100
700/700 [==============================] - 0s 30us/sample - loss: 4054.3056
Epoch 4/100
700/700 [==============================] - 0s 30us/sample - loss: 4005.4790
Epoch 5/100
700/700 [==============================] - 0s 27us/sample - loss: 3995.2293
Epoch 6/100
700/700 [==============================] - 0s 41us/sample - loss: 3996.1196
Epoch 7/100
700/700 [==============================] - 0s 41us/sample - loss: 3992.2517
Epoch 8/100
700/700 [==============================] - 0s 40us/sample - loss: 3994.0199
Epoch 9/100
700/700 [==============================] - 0s 31us/sample - loss: 3995.3253
Epoch 10/100
700/700 [==============================] - 0s 37us/sample - loss: 3995.1965
Epoch 11/100
700/700 [==============================] - 0s 34us/sample - loss: 3992.1188
Epoch 12/100
700/700 [=======

700/700 [==============================] - 0s 37us/sample - loss: 3995.1226
Epoch 94/100
700/700 [==============================] - 0s 36us/sample - loss: 3991.9554
Epoch 95/100
700/700 [==============================] - 0s 37us/sample - loss: 3993.8221
Epoch 96/100
700/700 [==============================] - 0s 34us/sample - loss: 3994.6647
Epoch 97/100
700/700 [==============================] - 0s 27us/sample - loss: 3994.7189
Epoch 98/100
700/700 [==============================] - 0s 29us/sample - loss: 3993.5228
Epoch 99/100
700/700 [==============================] - 0s 31us/sample - loss: 3992.5249
Epoch 100/100
700/700 [==============================] - 0s 29us/sample - loss: 3993.5534


In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.get_weights()

[array([[ 0.07153393],
        [-0.15705934],
        [ 0.7247697 ],
        [34.46843   ]], dtype=float32), array([34.460926], dtype=float32)]

### Classification using Keras 

In [36]:
pred = model.predict(x_test3)
pred

array([[68.92949 ],
       [68.92961 ],
       [68.92945 ],
       [68.92936 ],
       [68.929565],
       [68.92936 ],
       [68.92936 ],
       [68.92938 ],
       [68.92937 ],
       [68.92937 ],
       [68.929474],
       [68.929504],
       [68.92952 ],
       [68.92937 ],
       [68.929535],
       [68.92937 ],
       [68.9294  ],
       [68.92936 ],
       [68.92939 ],
       [68.92937 ],
       [68.92935 ],
       [68.92938 ],
       [68.929474],
       [68.92935 ],
       [68.92938 ],
       [68.929375],
       [68.9294  ],
       [68.92938 ],
       [68.92937 ],
       [68.92944 ],
       [68.92937 ],
       [68.92944 ],
       [68.92935 ],
       [68.92937 ],
       [68.92937 ],
       [68.92936 ],
       [68.92937 ],
       [68.92946 ],
       [68.92936 ],
       [68.92937 ],
       [68.92937 ],
       [68.92937 ],
       [68.92937 ],
       [68.92936 ],
       [68.92936 ],
       [68.92935 ],
       [68.9294  ],
       [68.92938 ],
       [68.92938 ],
       [68.92936 ],


### Load the given Iris data using pandas (Iris.csv)

In [58]:
iris = pd.read_csv('Iris.csv')

In [59]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [60]:
iris = iris.drop(['Id'], axis=1)

In [61]:
iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [62]:
# Assigning the label Column to targetVariable 

target = 'Species'

In [63]:
# Create Input Feature DataFrame X and Output Label Y for the Model Training

X_iris=iris.iloc[:,iris.columns!=target]
Y_iris=iris.iloc[:,iris.columns==target]

In [64]:
Y_iris.dtypes

Species    object
dtype: object

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [65]:
categorical_feature_mask = Y_iris.dtypes==object
categorical_cols = Y_iris.columns[categorical_feature_mask].tolist()

# import labelencoder
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()

# apply le on categorical feature columns
Y_iris[categorical_cols] = Y_iris[categorical_cols].apply(lambda col: le.fit_transform(col))

E:\Anaconda\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


### Divide the dataset into Training and test (70:30)

In [66]:
# Train Test Split

from sklearn.model_selection import train_test_split

np.random.seed(123)
x_train_iris,x_test_iris,y_train_iris,y_test_iris=train_test_split(X_iris,Y_iris,test_size=0.3,random_state=123)

In [67]:
y_train_iris = y_train_iris.to_numpy()

In [68]:
y_test_iris = y_test_iris.to_numpy()

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [69]:
#Initialize Sequential model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))
#model.add(tf.keras.layers.Dense(10), input_shape=(4,))

#Normalize the data


In [70]:
#First Dense Layer
model.add(tf.keras.layers.Dense(10))

In [71]:
#Add 1st hidden layer
model.add(tf.keras.layers.Dense(4, activation='sigmoid'))

In [72]:
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))

In [73]:
#Add OUTPUT layer
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [74]:
#Compile the model
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [75]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 4)                 16        
_________________________________________________________________
dense_5 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 44        
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 40        
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 27        
Total params: 177
Trainable params: 169
Non-trainable params: 8
_________________________________________________________________


### Fitting the model and predicting 

In [76]:
model.fit(x_train_iris,y_train_iris,          
          validation_data=(x_test_iris,y_test_iris),
          epochs=100,
          batch_size=32)

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 1s 6ms/sample - loss: 1.2445 - acc: 0.3143 - val_loss: 1.1716 - val_acc: 0.3778
Epoch 2/100
105/105 [==============================] - 0s 114us/sample - loss: 1.2270 - acc: 0.3143 - val_loss: 1.1652 - val_acc: 0.3778
Epoch 3/100
105/105 [==============================] - 0s 114us/sample - loss: 1.2125 - acc: 0.3143 - val_loss: 1.1589 - val_acc: 0.3778
Epoch 4/100
105/105 [==============================] - 0s 143us/sample - loss: 1.2021 - acc: 0.3143 - val_loss: 1.1533 - val_acc: 0.3778
Epoch 5/100
105/105 [==============================] - 0s 124us/sample - loss: 1.1880 - acc: 0.3143 - val_loss: 1.1470 - val_acc: 0.3778
Epoch 6/100
105/105 [==============================] - 0s 124us/sample - loss: 1.1768 - acc: 0.3143 - val_loss: 1.1428 - val_acc: 0.3778
Epoch 7/100
105/105 [==============================] - 0s 162us/sample - loss: 1.1638 - acc: 0.3143 - val_loss: 1.1384 - val_acc: 0.377

105/105 [==============================] - 0s 133us/sample - loss: 1.0519 - acc: 0.4857 - val_loss: 1.0887 - val_acc: 0.2444
Epoch 60/100
105/105 [==============================] - 0s 124us/sample - loss: 1.0524 - acc: 0.5238 - val_loss: 1.0880 - val_acc: 0.2667
Epoch 61/100
105/105 [==============================] - 0s 162us/sample - loss: 1.0519 - acc: 0.5333 - val_loss: 1.0874 - val_acc: 0.2667
Epoch 62/100
105/105 [==============================] - 0s 162us/sample - loss: 1.0514 - acc: 0.5905 - val_loss: 1.0852 - val_acc: 0.2889
Epoch 63/100
105/105 [==============================] - 0s 152us/sample - loss: 1.0525 - acc: 0.5238 - val_loss: 1.0823 - val_acc: 0.4444
Epoch 64/100
105/105 [==============================] - 0s 143us/sample - loss: 1.0486 - acc: 0.6667 - val_loss: 1.0810 - val_acc: 0.5111
Epoch 65/100
105/105 [==============================] - 0s 152us/sample - loss: 1.0499 - acc: 0.5714 - val_loss: 1.0804 - val_acc: 0.5111
Epoch 66/100
105/105 [=========================

In [77]:
pred = model.predict(x_test_iris)
pred

array([[0.26988256, 0.4088066 , 0.32131094],
       [0.25875154, 0.4232526 , 0.31799573],
       [0.25508106, 0.41886342, 0.32605556],
       [0.30149797, 0.37659132, 0.32191074],
       [0.36909887, 0.32476172, 0.30613944],
       [0.2737861 , 0.40188476, 0.3243292 ],
       [0.27394393, 0.40627408, 0.31978205],
       [0.3696694 , 0.31899235, 0.31133828],
       [0.36453798, 0.32105836, 0.31440362],
       [0.29351962, 0.37888247, 0.32759795],
       [0.2568666 , 0.42570207, 0.3174314 ],
       [0.3781445 , 0.33018762, 0.29166788],
       [0.29418266, 0.37309784, 0.33271948],
       [0.26734805, 0.41423965, 0.31841224],
       [0.25389215, 0.43384197, 0.31226584],
       [0.25679696, 0.4197903 , 0.3234128 ],
       [0.3702547 , 0.33345696, 0.29628837],
       [0.36935195, 0.3181063 , 0.31254175],
       [0.2852328 , 0.3964417 , 0.31832543],
       [0.36322328, 0.32622808, 0.31054863],
       [0.3705105 , 0.32851192, 0.3009776 ],
       [0.27540573, 0.4059084 , 0.31868583],
       [0.

### Report Accuracy of the predicted values

In [78]:
results = model.evaluate(x_test_iris, y_test_iris)

45/45 [==============================] - 0s 89us/sample - loss: 1.0410 - acc: 0.6222
